In [11]:
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import a_funciones as funciones  ###archivo de funciones propias
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import joblib  ### para guardar modelos
from sklearn.preprocessing import StandardScaler ## escalar variables 
import openpyxl
from sklearn.linear_model import LogisticRegression # Regresión logística
from sklearn.ensemble import RandomForestClassifier  # Clasificador bosques aleatoriost 
from sklearn.naive_bayes import GaussianNB


In [12]:
df_5 = pd.read_csv("https://raw.githubusercontent.com/andressj1/A-RH/main/Bases/df_final.csv", sep=",")
df_6 = pd.read_csv("https://raw.githubusercontent.com/andressj1/A-RH/main/Bases/base16.csv", sep=",")

## **Obtener dummies**

In [13]:
###### Convertimos las variables categorias a dummies
df_5d = pd.get_dummies(df_5)
df_5d

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,...,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,retirementType_Resignation,retirementType_no resignation,resignationReason_Others,resignationReason_Salary,resignationReason_Stress,resignationReason_no resignation
0,3.0,4.0,2.0,51,6,2,1,131160,1.0,11,...,0,0,1,0,0,1,0,0,0,1
1,3.0,2.0,4.0,31,10,1,1,41890,0.0,23,...,0,0,0,1,1,0,1,0,0,0
2,2.0,2.0,1.0,32,17,4,4,193280,1.0,15,...,0,0,1,0,0,1,0,0,0,1
3,4.0,4.0,3.0,38,2,5,3,83210,3.0,11,...,0,0,1,0,0,1,0,0,0,1
4,4.0,1.0,3.0,32,10,1,1,23420,4.0,12,...,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4293,4.0,1.0,3.0,42,5,4,1,60290,3.0,17,...,0,0,0,1,0,1,0,0,0,1
4294,4.0,4.0,3.0,29,2,4,1,26790,2.0,15,...,0,1,0,0,0,1,0,0,0,1
4295,1.0,3.0,3.0,25,25,2,2,37020,0.0,20,...,0,0,1,0,0,1,0,0,0,1
4296,4.0,1.0,3.0,42,18,2,1,23980,0.0,14,...,0,1,0,0,0,1,0,0,0,1


In [14]:
####Separamos nuestra variable objetivo de nuestra base final
X = df_5d.drop('Attrition',axis=1)
y = df_5d['Attrition']

print(X.shape)
print(y.shape)

(4298, 48)
(4298,)


In [16]:
y.head(2)

0    1
1    1
Name: Attrition, dtype: int64

## **Seleccion de variables**

In [ ]:
######Seleccion de modelos
### 1. Regresion logistica 
m_rl = LogisticRegression()
### 2. Random forest
m_rf = RandomForestClassifier()
### 3 Naive bayes
m_gn = GaussianNB()


In [ ]:
modelos = list([m_rl, m_rf, m_gn ])

In [ ]:
# Calcula el umbral como 2.5 veces la media de los valores
threshold = 2.5 * np.mean(X)

# Llama a la función sel_variables con el umbral calculado
var_names = funciones.sel_variables(modelos, X, y, threshold)

c:\Users\luisa\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1